## Detect cells per slice using Cellpose

#### Code created by Deniz Bekat (*deniz.bekat@crick.ac.uk*)

This code is designed to use standard `cellpose` datasets to measure the amount of detectable nuclei per slice in your images:
- uncleared images will *lose data* with iamging depth and less cells will be countable; a sharp **drop in cell count** is expected
- cleared images will *remain visible* and all cells can be counted; distribution of cells will depend on **the shape of your sample**


In this notebook we:
- import your image using `skimage.io.imread`
- download the nuclei cellpose model to measure cell count
- visualise your data and save it as a `.csv` file

### **Importing libraries**

Python contains several **modules and packages** that provide the tools to analyse our images; we can use the code ``` import ``` to use them within this code:

In [ ]:
#importing required modules

#modules for acessing and reading image files
import os


#modules for image analysis and data visualisation
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io
import pandas as pd

#modules for image visualisation and cell counting
import napari
from cellpose import models

### **Image and cellpose parameters**

We can specify the path for our image, and the parameters we will be using for counting cells using `cellpose`:

In [ ]:
#cellpose parameters; can change these depending on how the image looks (check in napari or Fiji first)

image_path = ""       #image path of your images to be analysed
n = 2       # process every nth slice
model_type = "nuclei"        # nuclear model for cell counting
diameter = None     # auto-scale by default; add a diameter in microns if necessary

### **Importing image**

We use the `scikit.io.imread`to import our test data! Note that this analysis will obviously use a nuclear marker. If you have a multichannel image, you can:
- convert the shape of your array with `np.tranpose`
- reduce the image to one channel with `imread = imread[n]`, where n is the channel number (keep in mind that Python counts from 0!)

In [ ]:
im_read = io.imread(path) #import the image with your given path
im_read.shape()     #shows the dimensions of the image array (e.g. TIF files will by CZXY); if you have more than one channel, you can reduce it down to just the nuclear channel

### **Visualise your data with `napari`**

The software `napari` is an open-source software that can be used to visualise your data in 2D and 3D; a quick qualitative check of your data is crucial alongside the quantitative analysis!

In [ ]:
viewer = napari.Viewer() #opens napari viewer
viewer.add_image(im_read)

### **Using cellpose to count nuclei**

[Cellpose](https://www.cellpose.org/) is an anatomical segmentation algorithm that can count nuclei with high precision. It is a machine-learning based algorithm similar to other algortihms such as `Stardist` but does not make assumptions about nuclei shape in this same way and can count condensed and irregular nuclei shapes much easier.

Here, we:
- load in cellpose to use
- use the algorithm to detect cells every `n` slices in your 3D image (where n is the variable you edited earlier in the code)

In [ ]:
#initialise and download cellpose model
model = models.CellposeModel(gpu=True)

In [ ]:
z_slices = range(0, im_read.shape[0], n) #counts every nth slices 
counts = []

for z in z_slices:
    img = im_read[z] 
    masks, flows, styles = model.eval(img, diameter=diameter)
    nuclei_count = len(np.unique(masks)) - 1  # counts the number of unique markers per slice; removes repeats and background (label 0)
    counts.append(nuclei_count)
    print(f"Slice {z}: {nuclei_count} nuclei") #shows the number of counts detected while the algorithm is running; if something seems off, you can stop this cell!

### **Organise and visualise your data**

Now we have this data in a Python `list`, we can put it together with other information, such as the image name, imaging system, and correlating your data with the slice index. 

After this, we can plot the data with `seaborn`, and show how your countable cells change with imaging depth!

In [ ]:
df = pd.DataFrame({
    "filename": [image_path],
    "slice_index": z_slices,
    "nuclei_count": counts
})
df['Type'] = '' #adds type of microscope to the dataframe for more controlled downstream analysis

df.tail()

In [ ]:
#plotting nuclei count against slices as a preliminary check of the data
sns.lineplot( 
    data=df,
    x='slice_index',
    y='nuclei_count',
    marker='o'
)


plt.xlabel('Slice index')
plt.ylabel('Nuclei count')
plt.grid(True)
plt.tight_layout()

### **Save your data**

We can then save your data in a `.csv` file; similar to your intensity measurements, you can combine different dataset together to visulise them for whatever your needs are!

In [ ]:
#export data to .csv files; several can be combined and then analysed in batch
df.to_csv('cellcount-results.csv')